<a href="https://colab.research.google.com/github/harrysillifant/pokerbots-2025/blob/main/CFRBountyTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Cython
!pip install eval7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 38.2 MB/s eta 0:00:00


In [ ]:
import eval7
from itertools import combinations
from itertools import combinations_with_replacement
from tqdm import tqdm
import random
import pickle
import numpy as np
import math
import time

In [ ]:
def save(ds, filename):
  with open(f"{filename}.pkl", "wb") as f:
    pickle.dump(ds, f)
  print(f"Saved to {filename}.pkl")

def load(filename):
  with open(f"{filename}.pkl", "rb") as f:
    ds = pickle.load(f)
  print(f"Loaded from {filename}.pkl")
  return ds

RANK_MAP = {
    0: '2', 1: '3', 2: '4', 3: '5', 4: '6', 5: '7', 6: '8', 7: '9',
    8: 'T', 9: 'J', 10: 'Q', 11: 'K', 12: 'A'
}
def is_premium(cards):
    """Premium hands: AA, KK, QQ, AKs"""
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    pair = cards[0].rank == cards[1].rank
    suited = cards[0].suit == cards[1].suit
    return (
        (pair and 'A' in ranks) or
        (pair and 'K' in ranks) or
        (pair and 'Q' in ranks) or
        (suited and {'A', 'K'} <= ranks)
    )

def is_strong(cards):
    """Strong hands: JJ, TT, 99, AQs, AJs, KQs, AQo, KQo"""
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    pair = cards[0].rank == cards[1].rank
    suited = cards[0].suit == cards[1].suit
    strong_pairs = {'J', 'T', '9'}
    strong_suited = [{'A', 'Q'}, {'A', 'J'}, {'K', 'Q'}]
    strong_offsuit = [{'A', 'Q'}, {'K', 'Q'}]
    return (
        (pair and ranks.pop() in strong_pairs) or
        (suited and ranks in strong_suited) or
        (not suited and ranks in strong_offsuit)
    )

def is_medium(cards):
    """Medium strength: 88, 77, 66, AJo, KJo, QJo, KJs, QJs, JTs"""
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    pair = cards[0].rank == cards[1].rank
    suited = cards[0].suit == cards[1].suit
    medium_pairs = {'8', '7', '6'}
    medium_suited = [{'K', 'J'}, {'Q', 'J'}, {'J', 'T'}]
    medium_offsuit = [{'A', 'J'}, {'K', 'J'}, {'Q', 'J'}]
    return (
        (pair and ranks.pop() in medium_pairs) or
        (suited and ranks in medium_suited) or
        (not suited and ranks in medium_offsuit)
    )

def is_low_pocket(cards):
    """Low pocket pairs: 55, 44, 33, 22"""
    return cards[0].rank == cards[1].rank and RANK_MAP[cards[0].rank] in {'5', '4', '3', '2'}

def is_high_ace(cards):
    """High Aces: ATs-A8s, ATo-A8o"""
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    high_aces = [{'A', 'T'}, {'A', '9'}, {'A', '8'}]
    return ranks in high_aces

def is_medium_suited_connector(cards):
    """Medium-Low Suited Connectors: T9s, 98s, 87s, 76s, 65s"""
    suited = cards[0].suit == cards[1].suit
    ranks = sorted([RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]], reverse=True)
    connectors = ['T9', '98', '87', '76', '65']
    return suited and ''.join(ranks) in connectors

def is_high_king_or_queen(cards):
    """High Kings/Queens: KTs, K9s, QTs, Q9s, KTo, QTo"""
    suited = cards[0].suit == cards[1].suit
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    high_suited = [{'K', 'T'}, {'K', '9'}, {'Q', 'T'}, {'Q', '9'}]
    high_offsuit = [{'K', 'T'}, {'Q', 'T'}]
    return (suited and ranks in high_suited) or (not suited and ranks in high_offsuit)

def is_low_suited_ace(cards):
    """Low Suited Aces: A7s-A2s"""
    suited = cards[0].suit == cards[1].suit
    ranks = {RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]}
    return suited and 'A' in ranks and ranks & {'7', '6', '5', '4', '3', '2'}

def is_low_suited_gapper(cards):
    """Low Suited Gappers: J9s, T8s, 97s, 86s, 75s"""
    suited = cards[0].suit == cards[1].suit
    ranks = sorted([RANK_MAP[cards[0].rank], RANK_MAP[cards[1].rank]], reverse=True)

    gappers = ['J9', 'T8', '97', '86', '75']
    return suited and ''.join(ranks) in gappers

def is_trash(cards):
    """Trash hands: Hands that don't fit into any other range."""
    return not (is_premium(cards) or is_strong(cards) or is_medium(cards) or
                is_low_pocket(cards) or is_high_ace(cards) or
                is_medium_suited_connector(cards) or
                is_high_king_or_queen(cards) or
                is_low_suited_ace(cards) or is_low_suited_gapper(cards))


class RangeGenerator:
    def __init__(self):
        self.ranges = {
        "Premium": [],
        "Strong": [],
        "Medium": [],
        "Low Pocket Pair": [],
        "High Ace": [],
        "Medium-Low Suited Connector": [],
        "High King/Queen": [],
        "Low Suited Ace": [],
        "Low Suited Gapper": [],
        "Trash": []
    }

        self.deck = list(eval7.Deck())
        self.holes = [c for c in combinations(self.deck, 2)]

        for hole_cards in self.holes:
            if is_premium(hole_cards):
                self.ranges["Premium"].append(hole_cards)
            elif is_strong(hole_cards):
                self.ranges["Strong"].append(hole_cards)
            elif is_medium(hole_cards):
                self.ranges["Medium"].append(hole_cards)
            elif is_low_pocket(hole_cards):
                self.ranges["Low Pocket Pair"].append(hole_cards)
            elif is_high_ace(hole_cards):
                self.ranges["High Ace"].append(hole_cards)
            elif is_medium_suited_connector(hole_cards):
                self.ranges["Medium-Low Suited Connector"].append(hole_cards)
            elif is_high_king_or_queen(hole_cards):
                self.ranges["High King/Queen"].append(hole_cards)
            elif is_low_suited_ace(hole_cards):
                self.ranges["Low Suited Ace"].append(hole_cards)
            elif is_low_suited_gapper(hole_cards):
                self.ranges["Low Suited Gapper"].append(hole_cards)
            else:
                self.ranges["Trash"].append(hole_cards)


# A dictionary mapping each rank character to its "poker value"
POKER_ORDER = {
    'A': 14,
    'K': 13,
    'Q': 12,
    'J': 11,
    'T': 10,
    '9': 9,
    '8': 8,
    '7': 7,
    '6': 6,
    '5': 5,
    '4': 4,
    '3': 3,
    '2': 2
}

def hole_pair_index(hole_pair):
    isomorphic_groups = generate_isomorphic_groups()

    rank1 = str(hole_pair[0]).upper()[0]
    rank2 = str(hole_pair[1]).upper()[0]
    suit1 = hole_pair[0].suit
    suit2 = hole_pair[1].suit

    # Compare based on poker values, not alphabetical
    if POKER_ORDER[rank1] > POKER_ORDER[rank2]:
        first, second = rank1, rank2
    else:
        first, second = rank2, rank1

    # Suited or offsuit
    if suit1 == suit2:
        hole_string = first + second + "s"
    else:
        hole_string = first + second + "o"
    # Now hole_string should match what generate_isomorphic_groups() produced
    return isomorphic_groups.index(hole_string)

def generate_isomorphic_groups():
    res = []
    cards = [str(i) for i in range(2, 10)] + ["T", "J", "Q", "K", "A"]
    cards = cards[::-1]
    combos = combinations_with_replacement(cards, 2)
    for cards in combos:
        hole = cards[0] + cards[1]
        if cards[0] != cards[1]:
            res.append(hole + "s")
        res.append(hole + "o")
    return res

allowed_pips = [1,2,4,6,8,10,12,15,18,20,25,30,50,100,150,200,250,300,350,400]
actions = ["f","ch","ca","sb","mb","lb","ai","r",None]

def encode_node(cluster, num_actions, last_action, pip):
    """
    Compute the local_index for (cluster, num_actions, last_action, pip) using allowed pips and actions.

    :param cluster:      Integer, 0..337 if street=0, else 0..149
    :param num_actions:  Integer, 0..3
    :param last_action:  String from actions or None
    :param pip:          Integer, one of the allowed pips
    :return:             local_index
    """

    # Validate inputs
    if pip not in allowed_pips:
        raise ValueError("Pip size not in allowed_pips.")
    if num_actions < 0 or num_actions > 3:
        raise ValueError("num_actions must be an integer between 0 and 3.")

    # Actions, extended to include None
    actions = ["f", "ch", "ca", "sb", "mb", "lb", "ai", "r"]
    extended_actions = actions + [None]

    if last_action not in extended_actions:
        raise ValueError("last_action must be a valid action from the actions list or None.")

    # Get the index of the last_action
    la_idx = extended_actions.index(last_action)  # Convert action string or None to index

    # Find the pip index in allowed_pips
    pip_idx = allowed_pips.index(pip)

    # Calculate the local_index
    local_index = (cluster * len(allowed_pips) * len(extended_actions) * 4) + \
                  (num_actions * len(allowed_pips) * len(extended_actions)) + \
                  (la_idx * len(allowed_pips)) + pip_idx

    return local_index

def decode_node(local_index):
    """
    Decode the local_index back into (cluster, num_actions, last_action, pip).

    :param local_index: Encoded index.
    :return:            A tuple (cluster, num_actions, last_action, pip).
    """

    # Actions, extended to include None
    actions = ["f", "ch", "ca", "sb", "mb", "lb", "ai", "r"]
    extended_actions = actions + [None]

    # Constants for decoding
    ACTION_SIZE = len(extended_actions)  # Includes None
    PIP_SIZE = len(allowed_pips)
    NUM_ACTIONS_SIZE = 4  # num_actions is in 0..3

    # Decode pip index
    pip_idx = local_index % PIP_SIZE
    local_index //= PIP_SIZE

    # Decode last_action index
    la_idx = local_index % ACTION_SIZE
    local_index //= ACTION_SIZE

    # Decode num_actions
    num_actions = local_index % NUM_ACTIONS_SIZE
    local_index //= NUM_ACTIONS_SIZE

    # Decode cluster
    cluster = local_index

    # Map pip_idx back to pip size
    pip = allowed_pips[pip_idx]

    # Map la_idx back to last_action string or None
    last_action = extended_actions[la_idx]

    return cluster, num_actions, last_action, pip

In [ ]:
encoded = encode_node(180, 2, "ca", 18)
print(encoded)
print(decode_node(encoded))

130008
(180, 2, 'ca', 18)


In [ ]:
deck = eval7.Deck()
seen = set()
for _ in range(50):
  deck.shuffle()
  hole = deck.peek(2)
  print(hole)
  out = hole_pair_index(hole)
  if out in seen:
    print("FOUND!")
    print(out)
  else:
    print(out)
    seen.add(out)


[Card("2d"), Card("6s")]
152
[Card("8s"), Card("2h")]
132
[Card("4c"), Card("Qs")]
64
[Card("5h"), Card("Kh")]
40
[Card("7d"), Card("7h")]
133
[Card("Jd"), Card("5h")]
81
[Card("6s"), Card("7s")]
134
[Card("5h"), Card("4h")]
154
[Card("9h"), Card("8s")]
107
[Card("9h"), Card("7s")]
109
[Card("Td"), Card("Ac")]
8
[Card("Ts"), Card("Th")]
88
[Card("Td"), Card("Jd")]
70
[Card("7c"), Card("8s")]
122
[Card("3d"), Card("8c")]
130
[Card("3d"), Card("4d")]
161
[Card("Jc"), Card("9s")]
73
[Card("4h"), Card("Ac")]
20
[Card("Tc"), Card("Kh")]
31
[Card("Js"), Card("5d")]
FOUND!
81
[Card("Ac"), Card("2d")]
24
[Card("Qc"), Card("7c")]
57
[Card("Kd"), Card("Qs")]
27
[Card("4d"), Card("4c")]
160
[Card("Kh"), Card("Qc")]
FOUND!
27
[Card("Ks"), Card("Jh")]
29
[Card("Kd"), Card("Ks")]
25
[Card("9s"), Card("Tc")]
90
[Card("7s"), Card("3h")]
141
[Card("Ad"), Card("7s")]
14
[Card("8h"), Card("Ah")]
11
[Card("Kh"), Card("5s")]
41
[Card("7c"), Card("Qc")]
FOUND!
57
[Card("6s"), Card("4h")]
148
[Card("Jc"), Ca

In [ ]:
# pot is at 12 postflop
# p1 smallbets (pot//3) -> temppot pot + pot//3
# p2 can either fold call or raise
# if p2 folds, pot is passed through as pot, and that player will lose the pot/2
# if p2 calls, pot is passed through as pot + 2*pot//3
# if p2 smallraises (newpot//3), pot is passed through as pot+2*pot//3 and if the next player wants to call, they need to put in newpot//3

In [ ]:
actions = ["f", "ch", "ca", "sb", "mb", "lb", "ai", "r", None]
pips = [1,2,4,6,8,10,12,15,18,20,25,30,50,100,150,200,250,300,350,400]
class BountyNode:
    def __init__(self, street, cluster, num_actions_at_street, last_action, other_pip, threshold=0.1):
        self.strategy = {}
        self.cum_strategy = {}
        self.cum_regret = {}
        self.threshold = threshold

        if last_action is None:
            if other_pip == 2 and street == 0: # sb acting at start of preflop
                for a in (0,2,3,4,5,6):
                    self.strategy[actions[a]] = 0
                    self.cum_strategy[actions[a]] = 0
                    self.cum_regret[actions[a]] = 0
            else: # Start of other streets, bb acts
                for a in (1,6):
                    self.strategy[actions[a]] = 0
                    self.cum_strategy[actions[a]] = 0
                    self.cum_regret[actions[a]] = 0
                allowed_bets = []
                if 2 <= other_pip + other_pip//2 < 400:
                    allowed_bets.append(3)
                elif 2 <= other_pip + other_pip < 400:
                    allowed_bets.append(4)
                elif 2 <= other_pip + 2*other_pip < 400:
                    allowed_bets.append(5)
                for a in allowed_bets: # add allin
                    self.strategy[actions[a]] = 0
                    self.cum_strategy[actions[a]] = 0
                    self.cum_regret[actions[a]] = 0

        elif last_action == "ch": # last player checked, can now check, bet(4)
            for a in (1,6): # all in and check always allowed
                self.strategy[actions[a]] = 0
                self.cum_strategy[actions[a]] = 0
                self.cum_regret[actions[a]] = 0
            if other_pip <= 10:
                  allowed_bets = [3,4,5,6]
            else:
                allowed_bets = []
                if 2 <= other_pip + other_pip//2 < 400:
                    allowed_bets.append(3)
                elif 2 <= other_pip + other_pip < 400:
                    allowed_bets.append(4)
                elif 2 <= other_pip + 2*other_pip < 400:
                    allowed_bets.append(5)
            for a in allowed_bets: # add allin
                self.strategy[actions[a]] = 0
                self.cum_strategy[actions[a]] = 0
                self.cum_regret[actions[a]] = 0

        elif last_action in ("sb","mb","lb"): # Opp bet, can reraise
            for a in (0,2,6): # Can always fold, call, allin
                self.strategy[actions[a]] = 0
                self.cum_strategy[actions[a]] = 0
                self.cum_regret[actions[a]] = 0
            if 2<= other_pip + other_pip < 400: # can legally raise
                self.strategy[actions[7]] = 0
                self.cum_strategy[actions[7]] = 0
                self.cum_regret[actions[7]] = 0

        elif last_action == "r": # Opp raised, can reraise if num_actions < 3
            for a in (0,2): # Can always fold or call
                self.strategy[actions[a]] = 0
                self.cum_strategy[actions[a]] = 0
                self.cum_regret[actions[a]] = 0
            if num_actions_at_street < 3:
                if 2<= other_pip + other_pip < 400: # can legally raise
                    self.strategy[actions[7]] = 0
                    self.cum_strategy[actions[7]] = 0
                    self.cum_regret[actions[7]] = 0
                self.strategy[actions[6]] = 0
                self.cum_strategy[actions[6]] = 0
                self.cum_regret[actions[6]] = 0


        elif last_action == "ai": # last player went all in, can now fold, call
            for a in (0,2):
                self.strategy[actions[a]] = 0
                self.cum_strategy[actions[a]] = 0
                self.cum_regret[actions[a]] = 0
        else:
            raise ValueError(f"Invalid last_action_taken: {last_action}")

        total = len(self.strategy.keys())
        for a in self.strategy.keys():
            self.strategy[a] = 1/total
            self.cum_strategy[a] = 0
            self.cum_regret[a] = 0

    def get_strategy(self, realization_weight):
        normalizing_factor = 0
        for a in self.strategy.keys():
            self.strategy[a] = max(0, self.cum_regret[a])
            normalizing_factor += self.strategy[a]
        for a in self.strategy.keys():
            if normalizing_factor == 0:
                self.strategy[a] = 1/len(self.strategy)
            else:
                self.strategy[a] /= normalizing_factor
        for a in self.strategy.keys():
            self.cum_strategy[a] += realization_weight*self.strategy[a]
        return self.strategy

    def get_avg_strategy(self):
        normalizing_factor = 0
        for a in self.strategy.keys():
            normalizing_factor += self.cum_strategy[a]
        for a in self.strategy.keys():
            if normalizing_factor == 0:
                self.cum_strategy[a] = 1/len(self.strategy)
            else:
                self.cum_strategy[a] /= normalizing_factor
        return self.cum_strategy

    def threshold_actions(self):
        to_del = []
        for a in self.strategy.keys():
            if self.strategy[a] <= self.threshold:
                to_del.append(a)
        for a in to_del:
            del self.strategy[a]
            del self.cum_strategy[a]
            del self.cum_regret[a]


# What if we have it such that the pot only updates after the bet has been matched GOOD IDEA

class BountyTrainer:
    def __init__(self, node_maps = None):
        if not node_maps:
            self.preflop_nodes = {}
            self.postflop_nodes = {}
            self.postturn_nodes = {}
            self.postriver_nodes = {}
            self.node_maps = [self.preflop_nodes, self.postflop_nodes, self.postturn_nodes, self.postriver_nodes]
        else:
            self.node_maps = node_maps
        range_generator = RangeGenerator()
        self.ranges = range_generator.ranges
        self.ranges_indexed = [x for x in self.ranges.values()]
        self.flop_centroids = load("/content/drive/MyDrive/flop_centroids")
        self.turn_centroids = load("/content/drive/MyDrive/turn_centroids")
        self.river_centroids = load("/content/drive/MyDrive/river_centroids")
        # self.allowed_pots = [3,4,5,6,10,15,30,50,75,100,200,300,400,500,600,700,800]

    def find_nearest_pip(self, other_pip):
        return min(pips, key=lambda pip: abs(pip - other_pip))

    def find_nearest_point(self, street, target):
        if street == 1:
            points = self.flop_centroids
        elif street == 2:
            points = self.turn_centroids
        elif street == 3:
            points = self.river_centroids
        # points is a 100x10 np array, target is a 1x10 np array
        distances = np.sum((points - target) ** 2, axis=1)
        nearest_index = np.argmin(distances)
        return nearest_index


    def get_cluster(self, hole, bounty, board,  mc_iters, opp_hand_samples=10):
        player_bounty_hit = (True if (bounty in [card.rank for card in hole]) or (bounty in [card.rank for card in board]) else False)
        if len(board) == 0:
            street = 0
        else:
            street = len(board)-2

        if street == 0: # Preflop, here clusters are a number from 0-168 representing the number of isomorphic holes
            cluster = hole_pair_index(hole)
            if player_bounty_hit:
                return cluster + 169
            else:
                return cluster

        point = []
        for ran in self.ranges_indexed:
            avg_equity = 0
            for _ in range(opp_hand_samples):
                opp_hand = list(random.choice(ran)) # Randomly picks a hand from the range
                wins = 0
                deck = eval7.Deck()
                deck.cards = [card for card in deck.cards if (card not in hole and card not in opp_hand and card not in board)]
                # Begin simulation for this our hole vs theirs with the flop down
                if street == 3: # River
                    hand_strength = eval7.evaluate(hole + board)
                    opp_strength = eval7.evaluate(opp_hand + board)
                    if hand_strength > opp_strength:
                        avg_equity += 1
                    elif hand_strength == opp_strength:
                        avg_equity += 0.5
                else:
                    for _ in range(mc_iters):
                        deck.shuffle()
                        full_board = board + deck.peek(5-len(board))
                        hand_strength = eval7.evaluate(hole + full_board)
                        opp_strength = eval7.evaluate(opp_hand + full_board)
                        if hand_strength > opp_strength:
                            wins += 1
                        elif hand_strength == opp_strength:
                            wins += 0.5
                    avg_equity += wins/mc_iters
            point.append(avg_equity/opp_hand_samples)

        # Now have point, which is a point in 10d space, now want to find the closest cluster
        cluster = self.find_nearest_point(street, np.array(point).reshape(1,10)) # returns index of the nearest cluster

        if player_bounty_hit:
            return cluster + 75
        else:
            return cluster


    def train(self,iters):
        def eval_showdown(player, opp, other_pip):
            my_hand = eval7.evaluate(player_holes[player]+board)
            their_hand = eval7.evaluate(player_holes[opp]+board)
            my_bounty_hit = (player_bounties[player] in [card.rank for card in player_holes[player]] or (player_bounties[player] in [card.rank for card in board]))
            opp_bounty_hit = (player_bounties[opp] in [card.rank for card in player_holes[player]] or (player_bounties[opp] in [card.rank for card in board]))
            if my_hand > their_hand:
                return other_pip*3/2 + 10 if my_bounty_hit else other_pip
            elif my_hand < their_hand:
                return -(other_pip*3/2 + 10) if opp_bounty_hit else -other_pip
            else: # draw
                if my_bounty_hit:
                    return other_pip/4 + 10
                elif opp_bounty_hit:
                    return -other_pip/4 - 10
                else:
                    return 0

        def cfr(street, history, p0, p1, path = set()): # History is a tuple of (num_actions_at_street, last_action, pip)
            num_actions,last_action,other_pip = history
            # print("cfr", street, history)
            if num_actions >= 4:
                raise KeyError("Too many actions!")
            if other_pip > 400:
                raise KeyError("Pip too big!")
            if street > 4:
                raise KeyError("Street too far!")
            if street == 0: # Preflop
                player = num_actions%2 # p0 is small blind here (they go first at start of game)
                opp = 1-player
            else:
                opp = num_actions%2
                player = 1-opp # p0 is small blind, but p1 (big blind) goes first in every other round

            if street == 4: # or (last_action is None and other_pip == 400): # Showdown
                return eval_showdown(player, opp, other_pip)

            cluster = player_clusters[player][street]
            other_pip = self.find_nearest_pip(other_pip) # Drag to closest allowed pip # THIS COULD CAUSE PROBLEMS

            # cluster, num_actions_at_street, last_action, other_pip
            id = encode_node(cluster, num_actions, last_action, other_pip)

            node = self.node_maps[street].get(id)
            if node is None:
                node = BountyNode(street, cluster, num_actions, last_action, other_pip)
                self.node_maps[street][id] = node


            # Forward recursive call to get utilites
            strategy = node.get_strategy(p0 if player==0 else p1)
            utility_for_action = {a:0 for a in strategy.keys()}
            node_utility = 0

            if other_pip <= 10: # Make sure small pots can be bet on
                sb = 4
                mb = 6
                lb = 10
                r = 10
            else:
                sb = other_pip//2
                mb = other_pip
                lb = 2*other_pip
                r = other_pip
                # print(r)


            # if (street,history) in path and not last_action == "ai":
            #     print(path)
            #     print(street, history)
            #     # Then we are in the same recursion chain -> cycle
            #     raise Exception("Infinite recursion cycle!")
            # path.add((street, history))

            for a in strategy.keys():
                if player == 0: # Small blind
                    if a == "f": # I fold, so I lose money, amplified if they had their bounty
                        if street == 0 and num_actions == 0: #First action is fold
                            utility_for_action[a] = -1
                        if last_action == "sb":
                            payout = 2*other_pip//3
                        elif last_action == "mb":
                            payout = other_pip//2
                        elif last_action == "lb":
                            payout = other_pip//3
                        elif last_action == "r":
                            payout = other_pip//2
                        elif last_action == "ai":
                            payout = other_pip
                        elif last_action is None:
                            if street == 0 and num_actions == 0:
                                payout = 1
                            else:
                                raise KeyError("Illegal fold move!")
                        else:
                            print(last_action)
                            raise KeyError("Illegal fold move!")

                        utility_for_action[a] = -payout*3/2 - 10 if (player_bounties[opp] in [card.rank for card in player_holes[opp]] or (player_bounties[opp] in [card.rank for card in board])) else -payout

                    elif a == "ch": # Check
                        if last_action == "ch": # opp checked
                            utility_for_action[a] = -cfr(street+1, (0, None, other_pip), p0*strategy[a], p1)
                        else: # player is first to act
                            if last_action is not None: raise KeyError("Illegal check move!")
                            utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip), p0*strategy[a], p1)

                    elif a == "ca": # Call
                        if street == 0 and last_action is None: # First action from sb
                            utility_for_action[a] = -cfr(street, (1, "ch", 2), p0*strategy[a], p1) # Interpret next opp infoset as a check from player
                        else: # Calling the other persons bet/raise, moves to next street, you match the money in their pot
                            if last_action == "ai":
                                utility_for_action[a] = eval_showdown(player, opp, 400)
                            else:
                                if last_action not in ("sb","mb","lb", "r"):
                                    print(last_action)
                                    raise KeyError("Call fucked!")
                                utility_for_action[a] = -cfr(street+1, (0, None, other_pip), p0*strategy[a], p1)


                    elif a == "sb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+sb), p0*strategy[a], p1) # 33% pot bet NOT
                    elif a == "mb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+mb), p0*strategy[a], p1) # 66% pot bet NOT
                    elif a == "lb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+lb), p0*strategy[a], p1) # 100% pot bet

                    elif a == "r":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip + r), p0*strategy[a], p1) # Raise them by double their money

                    elif a == "ai":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip), p0*strategy[a], p1)

                    else:
                        raise ValueError()

                else: # Big blind
                    if a == "f": # I fold, so I lose money, amplified if they had their bounty
                        if last_action == "sb":
                            payout = 2*other_pip//3
                        elif last_action == "mb":
                            payout = other_pip//2
                        elif last_action == "lb":
                            payout = other_pip//3
                        elif last_action == "r":
                            payout = other_pip//2
                        elif last_action == "ai":
                            payout = other_pip
                        else:
                            raise KeyError("Illegal fold move!")

                        utility_for_action[a] = -payout*3/2 - 10 if (player_bounties[opp] in [card.rank for card in player_holes[opp]] or (player_bounties[opp] in [card.rank for card in board])) else -payout

                    elif a == "ch": # Check
                        if last_action == "ch": # opp checked
                            utility_for_action[a] = -cfr(street+1, (0, None, other_pip), p0, p1*strategy[a])
                        else: # player is first to act
                            utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip), p0, p1*strategy[a])

                    elif a == "ca": # Call
                        if street == 0: # First action from sb, now bb is calling
                            utility_for_action[a] = cfr(street+1, (0, None, other_pip), p0, p1*strategy[a]) # Positive utility, since big blind takes next action
                        else: # Call their bet/raise on any other street
                            if last_action == "ai":
                                utility_for_action[a] = eval_showdown(player, opp, 400)
                            else:
                                if last_action not in ("sb","mb","lb", "r"):
                                    print(last_action)
                                    raise KeyError("Call fucked!")
                                utility_for_action[a] = -cfr(street+1, (0, None, other_pip), p0, p1*strategy[a])

                    elif a == "sb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+sb), p0, p1*strategy[a])
                    elif a == "mb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+mb), p0, p1*strategy[a])
                    elif a == "lb":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+lb), p0, p1*strategy[a])

                    elif a == "r":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip+r), p0, p1*strategy[a])

                    elif a == "ai":
                        utility_for_action[a] = -cfr(street, (num_actions+1, a, other_pip), p0, p1*strategy[a])

                    else:
                        raise ValueError()

            node_utility += strategy[a]*utility_for_action[a]

            for a in strategy.keys():
                regret = utility_for_action[a] - node_utility
                if player == 0:
                    node.cum_regret[a] += p1*regret
                else:
                    node.cum_regret[a] += p0*regret

            return node_utility




        net_util_for_p0 = 0
        deck = eval7.Deck()
        for i in tqdm(range(1, iters+1)):
            deck.shuffle()
            cards = deck.peek(11)
            p0_hole = cards[:2]
            p1_hole = cards[2:4]
            board = cards[4:]
            ranks = [i for i in range(0,13)]
            p0_bounty = random.choice(ranks)
            p1_bounty = random.choice(ranks)
            player_holes = [p0_hole, p1_hole]
            player_bounties = [p0_bounty, p1_bounty]

            p0_clusters = [self.get_cluster(p0_hole, p0_bounty, board[:(i+2 if i > 0 else i)], (20 if i == 1 else 10), (15 if i == 1 else 10)) for i in range(4)]
            p1_clusters = [self.get_cluster(p1_hole, p1_bounty, board[:(i+2 if i >= 0 else i)], (20 if i == 1 else 10), (15 if i == 1 else 10)) for i in range(4)]
            player_clusters = [p0_clusters, p1_clusters]

            net_util_for_p0 += cfr(0, (0,None,2), 1, 1)
            if i == 9*iters//10 and i % 500 == 0: # Threshold actions
                print("THRESHOLDED")
                for node_map in self.node_maps:
                    for id in node_map.keys():
                          node = node_map.get(id)
                          if node is not None:
                              node.threshold_actions()


            if i % 10000 == 0:
                print("Avg Utility for Small Blind: ", net_util_for_p0/iters)
                save(self.node_maps, f"/content/drive/MyDrive/bountystrats3/strategy{i}")

            if i == 100000: # Reset cumulative strategy, to scrap early errors
                for node_map in self.node_maps:
                    for id in node_map.keys():
                        node = node_map.get(id)
                        if node is not None:
                            for a in node.strategy.keys():
                                node.cum_strategy[a] = 0




In [ ]:
# import sys

# # Get the current recursion limit
# print("Current recursion limit:", sys.getrecursionlimit())

# # Set a new recursion limit
# sys.setrecursionlimit(10000)  # Set the limit to a higher value, e.g., 10,000

# # Verify the new recursion limit
# print("New recursion limit:", sys.getrecursionlimit())

In [ ]:
trainer = BountyTrainer()
trainer.train(250000)
node_maps = trainer.node_maps
for street in range(4):
  for id in node_maps[street].keys():
    node = node_maps[street].get(id)
    if node is not None:
      print([node.get_avg_strategy()[a] for a in node.strategy.keys()])
for street in range(4):
  print(len(node_maps[street]))

Loaded from /content/drive/MyDrive/flop_centroids.pkl
Loaded from /content/drive/MyDrive/turn_centroids.pkl
Loaded from /content/drive/MyDrive/river_centroids.pkl


  0%|          | 19/250000 [00:05<19:27:27,  3.57it/s]


KeyboardInterrupt: 

CONVERGED UNTIL THRESHOLDING THEN MOVED AWAY @ iteration 25000 -> mustve cooked it

In [ ]:
def convert(node_maps):
    res = {}
    for street in range(4):
      for id in node_maps[street].keys():
        node = node_maps[street].get(id)
        strat = node.get_avg_strategy()
        res[id] = [(strat[a] if a in node.valid_actions else 0) for a in range(7)]
    return res

In [ ]:
node_maps = load("/content/drive/MyDrive/bountystrats2/strategy300000")
trainer = BountyTrainer(node_maps)
trainer.train(1000000)
node_maps = trainer.node_maps
for street in range(4):
  for id in node_maps[street].keys():
    node = node_maps[street].get(id)
    if node is not None:
      print([node.get_avg_strategy()[a] for a in node.strategy.keys()])
for street in range(4):
  print(len(node_maps[street]))

Loaded from /content/drive/MyDrive/bountystrats2/strategy300000.pkl
Loaded from /content/drive/MyDrive/flop_centroids.pkl
Loaded from /content/drive/MyDrive/turn_centroids.pkl
Loaded from /content/drive/MyDrive/river_centroids.pkl


  1%|          | 9999/1000000 [46:00<75:26:57,  3.64it/s]

Avg Utility for Small Blind:  -6.84950933191338e-05


  1%|          | 10000/1000000 [46:01<97:16:50,  2.83it/s]

Saved to /content/drive/MyDrive/bountystrats3/strategy10000.pkl


...

 29%|██▉       | 289999/1000000 [22:20:27<54:37:13,  3.61it/s]

Avg Utility for Small Blind:  -0.00026658063658779575


 29%|██▉       | 290000/1000000 [22:20:27<69:47:02,  2.83it/s]

Saved to /content/drive/MyDrive/bountystrats3/strategy290000.pkl


 29%|██▉       | 292381/1000000 [22:31:33<53:56:08,  3.64it/s]

In [ ]:
converted_maps = convert(node_maps)
print(converted_maps)

{305634: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 128479: [0, 0.0, 0, 0.0, 0.0, 0.0, 1.0], 302443: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 130077: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 130875: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 131673: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 132471: [0.5, 0, 0.5, 0, 0, 0, 0], 303241: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 130086: [0.16666666666666666, 0, 0.16666666666666666, 0.16666666666666666, 0.166666666666

In [ ]:
print(decode_node(128479))

(20, 1, 4)


In [ ]:
print(decode_node(952015))

(149, 1, 4)
